# Modeling Covid Cases and Deaths with Python by Ted Petrou

**Topics**

* Dashboard Overview
* Data Smoothing
* Modeling with "S"-shaped curves
* Ensemble Modeling

## Dashboard Overview

* https://coronavirus.dunderdata.com

## Data Source

* https://github.com/CSSEGISandData/COVID-19
* cleaned
* cumulative

### Read in last 150 days of USA cases

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dashboard.mplstyle')
pd.set_option('display.max_columns', None)

usa_cases = pd.read_csv('data/usa_cases.csv').iloc[-150:]
usa_cases.tail(3)

# Data Smoothing

In [ ]:
texas = usa_cases['Texas']
texas.tail(3)

### Cumulative graph

* looks smooth

In [ ]:
texas.plot();

### Daily graph

* not smooth
* reporting oddities

In [ ]:
texas_daily = texas.diff().dropna()
texas_daily.plot(title='Texas Daily Cases');

### Smoothing techniques

* rolling average
* lowess

In [ ]:
texas_daily.plot(label='actual');
texas_smoothed = texas_daily.rolling(7, min_periods=1, center=True).mean()
texas_smoothed.plot(title='Smoothed Texas Daily Cases', label='smoothed');

In [ ]:
for i in range(3):
    texas_smoothed = texas_smoothed.rolling(7, min_periods=1, center=True).mean()

texas_daily.plot(label='actual')
texas_smoothed.plot(title='Texas Daily Cases - Repeated Rolling Average Smoothed', label='smoothed').legend();

In [ ]:
texas_cumulative_smoothed = texas_smoothed.cumsum() + texas.iloc[0]
texas.plot(label='actual', title='Texas Cumulative Cases - Rolling Average Smoothed')
texas_cumulative_smoothed.plot(label='smoothed').legend();

### Lowess

Locally weighted scatter plot smoothing

* A low-degree polynomial regression line is fit to each point using the nearest (local) points
* Choose number of points with `frac`

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess

n = len(texas_daily)
x = np.arange(n)
y = texas_daily.values

frac = 21 / n
y_lowess = lowess(y, x, frac=frac, return_sorted=False)
texas_daily_lowess = pd.Series(y_lowess, index=texas_daily.index)
texas_daily.plot(label='actual')
texas_daily_lowess.plot(label='smoothed', title="Texas Daily Cases - Lowess Smooothed").legend();

In [ ]:
texas_cumulative_lowess = texas_daily_lowess.cumsum() + texas.iloc[0]
texas.plot(label='actual')
texas_cumulative_lowess.plot(label='smoothed', title="Texas Cumulative Cases - Lowess Smooothed").legend();

## Modeling with "S"-shaped curves

* Only modeling with historical data
* Modeling cumulative
* [Sigmoid functions](https://en.wikipedia.org/wiki/Sigmoid_function)

$$f(x) = \frac{1}{1 + e^x}$$

In [ ]:
x = np.linspace(-5, 5, 50)
y = 1 / (1 + np.exp(x))
fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_title('Simple "S"-Curve - Logistic Function');

Correct shape, needs to be transposed around y axis

$$f(x) = \frac{1}{1 + e^{-x}}$$

In [ ]:
x = np.linspace(-5, 5, 50)
y = 1 / (1 + np.exp(-x))
plt.plot(x, y)
plt.title('Logistic function transposed around y-axis');

Increase maximum

$$f(x) = \frac{L}{1 + e^{-x}}$$

In [ ]:
L = 10_000
x = np.linspace(-5, 5, 50)
y = L / (1 + np.exp(-x))
plt.plot(x, y)
plt.title(f'Logistic Function with Asymptote at {L:,}');

Allow horizontal shift, $x_0$ is middle, inflection point.

$$f(x) = \frac{L}{1 + e^{-(x - x_0)}}$$

In [ ]:
L = 10_000
x0 = 50
x = np.linspace(40, 60)
y = L / (1 + np.exp(-(x - x0)))
plt.plot(x, y)
plt.title('Logistic Function - Scaled and Shifted');

Allow for different growth rate (steepness)

$$f(x) = \frac{L}{1 + e^{-k(x - x_0)}}$$

In [ ]:
L = 10_000
x0 = 50
k = 0.06
x = np.linspace(0, 100, 50)
y = L / (1 + np.exp(-k * (x - x0)))
plt.plot(x, y)
plt.title('Logistic Function - Scaled, Shfited, and Flattened');

In [ ]:
def logistic(x, L, x0, k):
    return L / (1 + np.exp(-k * (x - x0)))

In [ ]:
x = np.linspace(0, 500)
y = logistic(x, 100_000, 120, .01)
plt.plot(x, y);

### Manually fit logistic function to data?

In [ ]:
y_texas = texas_cumulative_lowess
def manual_fit(y, func, params):
    n = len(y)
    x = np.arange(n)
    y_model = func(x, *params)
    y_model = pd.Series(y_model, index=y.index)
    y.plot(label='actual')
    y_model.plot(label='model').legend()

In [ ]:
# manual_fit()

### Scipy's optimize module

* Use `least_squares` function - finds parameters that minimize squared residuals
* Create optimization function that computes residuals
* Must create bounds for parameters
* Must have initial guess for parameters

In [ ]:
def optimize_func(params, x, y, model):
    y_pred = model(x, *params)
    error = y - y_pred
    return error

In [ ]:
from scipy.optimize import least_squares

# bounds for parameters L, x0, and k
lower = y_texas.iloc[-1], 20, 0.01
upper = 4_000_000, 150, 0.5
bounds = lower, upper

# initial guess for L, x0, and k
p0 = 3_000_000, 50, 0.1

# run with verbose
y = y_texas.values
x = np.arange(len(y))
res = least_squares(optimize_func, p0, args=(x, y, logistic), bounds=bounds, verbose=2)

Fitted parameters are in `x` attribute

In [ ]:
res.x

In [ ]:
manual_fit(y_texas, logistic, res.x)

## Generalized Logistic Function

* [Asymmetric logistic function][0] - tail slowly trail off
* Also allow for vertical shift


$$f(x) = \frac{L - s}{(1 + e^{-kx})^{\frac{1}{v}}} + s$$

[0]: https://en.wikipedia.org/wiki/Generalised_logistic_function

In [ ]:
def general_logistic_shift(x, L, k, v, s):
    return (L - s) / ((1 + np.exp(-k * x)) ** (1 / v)) + s

# bounds
lower = y_texas.iloc[-1], 0.01, 0.01, 0
upper = 4_000_000, 0.5, 1, y_texas.iloc[-1]
bounds = lower, upper

# initial guess for L, k, v, s
p0 = 3_000_000, 0.1, 0.1, y_texas.iloc[-1]

# run with verbose
y = y_texas.values
x = np.arange(len(y))
res = least_squares(optimize_func, p0, args=(x, y, general_logistic_shift), bounds=bounds, verbose=2)

In [ ]:
res.x

In [ ]:
manual_fit(y_texas, general_logistic_shift, res.x)

## Modeling Deaths

* Could do same thing
* Use historical case fatality rate (fraction of cases that lead to death)

## Ensembling and modeling new waves

* Modeling new waves
    * Could model `L` as a Logistic function itself (difficult)
    * Only allow model to see n previous days
    * Choose many different values of n and average result